### Setup 

In [1]:
import sys
# Python 3.7 is required
assert sys.version_info >= (3,7)

import cv2 as cv
import numpy as np

# For reproducibility,
np.random.seed(99)

# Make sure that optimization is enabled
if not cv.useOptimized():
    cv.setUseOptimized(True)

cv.useOptimized()

True

### Question 1 

In [2]:
def crop_grid(img, num_horizontal_grid, num_vertical_grid, line_color):
    ori_img = cv.imread(img)
    img_copy = ori_img.copy()
    
    height, width = ori_img.shape[:2]
    M, N = int(height/num_horizontal_grid), int(width/num_vertical_grid)
    
    x1, y1 = 0, 0

    for y in range(0, height, M):
        for x in range(0, width, N):
            y1 = y + M    # lower right coordinate that will be used to construct rectangle
            x1 = x + N
            # Check whether patch lower right coordinate exceeds image height and width
            if x1 >= width and y1 >= height:
                x1 = width - 1
                y1 = height - 1
                tile = ori_img[y:height, x:width]
                cv.rectangle(img_copy, (x, y), (x1, y1), line_color, 1)
            # When patch lower right y-coordinate exceeds patch height
            elif y1 >= height:
                y1 = height - 1
                cv.rectangle(img_copy, (x, y), (x1, y1), line_color, 1)

            # When patch lower right x-coordinate exceeds patch width
            elif x1 >= width:
                x1 = width - 1
                cv.rectangle(img_copy, (x, y), (x1, y1), line_color, 1)

            else:
                cv.rectangle(img_copy, (x, y), (x1, y1), line_color, 1)
    return cv.imshow('patched image', img_copy)

In [3]:
crop_grid('dog.jfif', 5, 10, (0,0,255))
cv.waitKey(0)
cv.destroyAllWindows()

### Question 2 

In [8]:
img1 = cv.imread('lena.jfif')
img2 = cv.imread('coins.jfif')

# Resize img2
new_shape = img1.shape[:2]
img2 = cv.resize(img2, new_shape)

# show 3 images with different random value of alpha 
for i in np.arange(0,3): 
    alpha = np.array((round(np.random.uniform(0,1),2))) # generate three random alpha values
    dst = cv.addWeighted(img1, float(alpha), img2, 1 - alpha, 0) # Beta value always 1 - alpha
    cv.putText(dst, "a={}".format(alpha),(24,22), 0, 0.5, 0,1, cv.LINE_AA) # Show the alpha value
    cv.imshow('Image Blendingz', dst)
    cv.waitKey(0)
    cv.destroyAllWindows()

### Question 3

$$\begin{bmatrix} \alpha & \beta & (1 - \alpha)\cdot c_{x} - \beta \cdot c_{y}\\ -\beta & \alpha &  \beta \cdot c_{x} + (1 - \alpha)\cdot c_{y} \end{bmatrix}$$

In [102]:
img = cv.imread('lena.jfif')

rows, cols = img.shape[:2]
center = ((cols-1)/2.0,(rows-1)/2.0)
angle = 45
M = cv.getRotationMatrix2D(((cols-1)/2.0, (rows-1)/2.0), angle, 1)

# the location of alpha and beta value in M is row 0 with col 0 and 1
cosValue = np.abs(M[0][0])
sinValue = np.abs(M[0][1])

# calculate the new height and width bouds after the iamge rotate
newRotateHeight = int((cols * sinValue) + (rows * cosValue))
newRotateWidth = int((cols * cosValue) + (rows * sinValue))

# subtract the old image center and adding the new image center
M[0][2] += (newRotateWidth/2) - center[1]
M[1][2] += (newRotateHeight/2) - center[0]

dst = cv.warpAffine(img, M, (newRotateHeight,newRotateWidth))
cv.imshow('Image Blendingz', dst)
cv.waitKey(0)
cv.destroyAllWindows()

### Question 4

In [96]:
# Load two images
img1 = cv.imread('flower.jfif')
img2 = cv.imread('native-bee.png')

# Create ROI from the flower 
rows, cols =  img1.shape[:2]
ROI = img2[0:rows, 0:cols]

# Create a mask of the flower and its inverse mask
flowerGray = cv.cvtColor(img1, cv.COLOR_BGR2GRAY)
ret, mask = cv.threshold(flowerGray,70 ,255, cv.THRESH_BINARY)
maskInv = cv.bitwise_not(mask)

# Black out the are of the flower in ROI
img2_bg = cv.bitwise_and(ROI,ROI,mask = maskInv)

#Take the region of the flower
img1_fg = cv.bitwise_and(img1,img1,mask = mask)

# Put flower in ROI
dst = cv.add(img1_fg,img2_bg)
img2[0:rows, 0:cols] = dst


cv.imshow('flower_bee',img2)
cv.waitKey(0)
cv.destroyAllWindows()